# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [2]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [40]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    #TODO 1: Add your code here
    header = ["Title", "Year", "Synopsis", "Review"]

    df_eng = pd.read_csv("movie_reviews_eng.csv",
                       delimiter=",",
                       names=header
                       )
    df_eng = df_eng.drop(index=0)
    df_eng["Original Language"] = "English"

    df_fr = pd.read_csv("movie_reviews_fr.csv",
                       delimiter=",",
                       names=header
                       )
    df_fr = df_fr.drop(index=0)
    df_fr["Original Language"] = "French"

    df_sp = pd.read_csv("movie_reviews_sp.csv",
                       delimiter=",",
                       names=header
                       )
    df_sp = df_sp.drop(index=0)
    df_sp["Original Language"] = "Spanish"

    df = pd.concat([df_eng, df_fr], ignore_index=True)
    df = pd.concat([df, df_sp], ignore_index=True)

    return df

df = preprocess_data()
assert df.shape == (30, 5), "Error: Shape of read dataframe does not match the expected shape!"

In [23]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",Spanish
20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",Spanish
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French
10,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",French
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",Spanish


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [6]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)

es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)


# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

(…)LP/opus-mt-fr-en/resolve/main/source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

(…)LP/opus-mt-fr-en/resolve/main/target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

(…)LP/opus-mt-fr-en/resolve/main/vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

(…)fr-en/resolve/main/tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

/Users/anthony/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


...

(…)s-en/resolve/main/generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

(…)LP/opus-mt-es-en/resolve/main/source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

(…)LP/opus-mt-es-en/resolve/main/target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

(…)LP/opus-mt-es-en/resolve/main/vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

(…)es-en/resolve/main/tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

In [41]:
# TODO 4: Update the code below

# Translate the respective languages and overwrite the existing data
headers = ["Review", "Synopsis"]
for header in headers:
    df[header] = df.apply(lambda row: translate(row[header], fr_en_model, fr_en_tokenizer) if row['Original Language'] == "French" else row[header], axis=1)

for header in headers:
    df[header] = df.apply(lambda row: translate(row[header], es_en_model, es_en_tokenizer) if row['Original Language'] == "Spanish" else row[header], axis=1)

In [42]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [50]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "text-classification"
sentiment_classifier = pipeline("text-classification")

# only for testing
print(sentiment_classifier("I love you"))

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    results = sentiment_classifier(text)
    sentiment= results[0]["label"]
    return sentiment

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763}]


In [51]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["Sentiment"] = df.apply(lambda row: analyze_sentiment(row["Review"], sentiment_classifier), axis=1)



In [52]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French,NEGATIVE
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English,NEGATIVE
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English,POSITIVE
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",French,NEGATIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish,NEGATIVE
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,NEGATIVE
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,POSITIVE
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish,POSITIVE
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,NEGATIVE
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish,NEGATIVE


In [54]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)